# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [4]:
# Write your code below.
%reload_ext dotenv
%dotenv


In [5]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [28]:
import os
from glob import glob
import dask.dataframe as dd

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(os.getenv("PRICE_DATA"), "*/*.parquet/*.*.parquet"))
dd_px = dd.read_parquet(parquet_files).set_index("ticker")
print(dd_px.head())

             Date       Open       High        Low      Close  Adj Close  \
ticker                                                                     
A      2000-01-03  56.330471  56.464592  48.193848  51.502148  43.613007   
A      2000-01-04  48.730328  49.266811  46.316166  47.567955  40.281456   
A      2000-01-05  47.389126  47.567955  43.141991  44.617310  37.782803   
A      2000-01-06  44.080830  44.349072  41.577251  42.918453  36.344173   
A      2000-01-07  42.247852  47.165592  42.203148  46.494991  39.372841   

         Volume       sector                       subsector  year  
ticker                                                              
A       4674353  Health Care  Life Sciences Tools & Services  2000  
A       4765083  Health Care  Life Sciences Tools & Services  2000  
A       5758642  Health Care  Life Sciences Tools & Services  2000  
A       2534434  Health Care  Life Sciences Tools & Services  2000  
A       2819626  Health Care  Life Sciences Tools & S

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [31]:
# Write your code below.
dd_feat = dd_px.map_partitions(
    lambda df: df.assign(
        Close_lag=df['Close'].shift(1),
        returns=(df['Close'] / df['Close'].shift(1)) - 1,
        hi_lo_range=df['High'] - df['Low']
    )
)

# Step 4: Compute and display results
dd_feat = dd_feat.compute()
print(dd_feat.head())

             Date       Open       High        Low      Close  Adj Close  \
ticker                                                                     
A      2000-01-03  56.330471  56.464592  48.193848  51.502148  43.613007   
A      2000-01-04  48.730328  49.266811  46.316166  47.567955  40.281456   
A      2000-01-05  47.389126  47.567955  43.141991  44.617310  37.782803   
A      2000-01-06  44.080830  44.349072  41.577251  42.918453  36.344173   
A      2000-01-07  42.247852  47.165592  42.203148  46.494991  39.372841   

         Volume       sector                       subsector  year  Close_lag  \
ticker                                                                          
A       4674353  Health Care  Life Sciences Tools & Services  2000        NaN   
A       4765083  Health Care  Life Sciences Tools & Services  2000  51.502148   
A       5758642  Health Care  Life Sciences Tools & Services  2000  47.567955   
A       2534434  Health Care  Life Sciences Tools & Services  

+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [33]:
# Write your code below.
dd_feat['rolling_avg_return'] = dd_feat.groupby('ticker')['returns'].rolling(window=10).mean().reset_index(level=0, drop=True)
print(dd_feat.head())


             Date       Open       High        Low      Close  Adj Close  \
ticker                                                                     
A      2000-01-03  56.330471  56.464592  48.193848  51.502148  43.613007   
A      2000-01-04  48.730328  49.266811  46.316166  47.567955  40.281456   
A      2000-01-05  47.389126  47.567955  43.141991  44.617310  37.782803   
A      2000-01-06  44.080830  44.349072  41.577251  42.918453  36.344173   
A      2000-01-07  42.247852  47.165592  42.203148  46.494991  39.372841   

         Volume       sector                       subsector  year  Close_lag  \
ticker                                                                          
A       4674353  Health Care  Life Sciences Tools & Services  2000        NaN   
A       4765083  Health Care  Life Sciences Tools & Services  2000  51.502148   
A       5758642  Health Care  Life Sciences Tools & Services  2000  47.567955   
A       2534434  Health Care  Life Sciences Tools & Services  

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?

Converting to Pandas was necessary to calculate the rolling average return because Dask does not directly support rolling calculations like Pandas does. Dask's design focuses on parallelizing operations across multiple cores or machines, which often sacrifices support for certain operations that are not easily parallelizable.

+ Would it have been better to do it in Dask? Why?
No. Implementing rolling calculations directly in Dask would involve custom functions or potentially less efficient workarounds. This could introduce complexity and reduce clarity in the code, especially for operations that are straightforward in Pandas. Using Pandas for specific operations like rolling calculations is often the preferred approach for simplicity and performance.


(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.